# Load saved model

In [3]:
from keras.models import load_model
import numpy as np

In [2]:
model = load_model('cnn_delta_80_640.h5')

In [4]:
data = np.load('test_delta_data.npy')

In [5]:
data.shape

(200, 80, 640, 1)

In [6]:
model.predict(data[:10])

array([[ 0.1976981 ],
       [ 0.38741246],
       [ 0.23059584],
       [ 0.55642492],
       [ 0.53885239],
       [ 0.53384912],
       [ 0.13143235],
       [ 0.36180514],
       [ 0.25996241],
       [ 0.6673317 ]], dtype=float32)

In [9]:
from keras import backend as K

import tensorflow as tf
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib

In [39]:
def export_model(saver, model, MODEL_NAME, input_node_names = None, output_node_name = None):
    
    if input_node_names is None:
        input_node_names = [x.op.name for x in model.inputs]
    if output_node_name is None:
        output_node_name = [x.op.name for x in model.outputs][0]
    
    tf.train.write_graph(K.get_session().graph_def, 'out', \
        MODEL_NAME + '_graph.pbtxt')

    saver.save(K.get_session(), 'out/' + MODEL_NAME + '.chkp')

    freeze_graph.freeze_graph('out/' + MODEL_NAME + '_graph.pbtxt', None, \
        False, 'out/' + MODEL_NAME + '.chkp', output_node_name, \
        "save/restore_all", "save/Const:0", \
        'out/frozen_' + MODEL_NAME + '.pb', True, "")

    input_graph_def = tf.GraphDef()
    with tf.gfile.Open('out/frozen_' + MODEL_NAME + '.pb', "rb") as f:
        input_graph_def.ParseFromString(f.read())

    output_graph_def = optimize_for_inference_lib.optimize_for_inference(
            input_graph_def, input_node_names, [output_node_name],
            tf.float32.as_datatype_enum)

    with tf.gfile.FastGFile('out/opt_' + MODEL_NAME + '.pb', "wb") as f:
        f.write(output_graph_def.SerializeToString())

    print("graph saved!")

save the model like the following:
```
export_model(tf.train.Saver(), model, ["conv2d_1_input"], "dense_2/Softmax")
```

In [40]:
export_model(tf.train.Saver(), model, 'cnn_delta_80_640_graph')

INFO:tensorflow:Froze 10 variables.
Converted 10 variables to const ops.
107 ops in the final graph.
graph saved!
